<a href="https://colab.research.google.com/github/kslim1025/TF_Keras_RNN/blob/master/RNN_KoreanLang_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
# MNIST 문자열 분석으로 유명한 데이터셋이 패션에 관한 데이터 세트를 만듬 그게 FashionMINIST 데이터셋이다.
# 데이터 이미지가 0에서 255까지 값을 가지는 28x28이미지라는 것을 확인가능
# 정답이 되는 라벨을 확인하기 위해 print를 붙여서  확인
# 외부 데이터를 이용한 정제과정
# ctrl+enter를 사용한 런타임가능
# !nvidia-smi : 어떤 GPU를 사용하는지 확인가능한 명령어 
#

try:
  # %tensorflow_version only exists in Colab
  %tensorflow_version 2.x
except Exception:
  pass

# 넘파이는 수학과 과학 연산에 특화된 파이썬 모듈로 딥러닝에서도 유용하게 사용된다.

import numpy as np;
import tensorflow as tf;
import pandas as pd;
import matplotlib.pyplot as plt;
import math;
import re;
################################################################################
!nvidia-smi
       
#+-----------------------------------------------------------------------------+
#| NVIDIA-SMI 450.66       Driver Version: 418.67       CUDA Version: 10.1     |
#|-------------------------------+----------------------+----------------------+
#| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
#| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
#|                               |                      |               MIG M. |
#|===============================+======================+======================|
#|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
#| N/A   51C    P0    35W / 250W |   1581MiB / 16280MiB |      0%      Default |
#|                               |                      |                 ERR! |
#+-------------------------------+----------------------+----------------------+
#                                                                               
#+-----------------------------------------------------------------------------+
#| Processes:                                                                  |
#|  GPU   GI   CI        PID   Type   Process name                  GPU Memory |
#|        ID   ID                                                   Usage      |
#|=============================================================================|
#|  No running processes found                                                 |
#+-----------------------------------------------------------------------------+

################################################################################

# 한글 원본 텍스트를자소 단위와 단어 단위로 나눠서 순환 신경망으로 생성
# 이번에 작업할때 사용할 원본 텍스트는 국사 편찬위원회에서 제공하는 조선왕조실록 국문 번역본을 사용

path_to_file = tf.keras.utils.get_file('input.txt', 'http://bit.ly/2Mc3SOV')

#데이터를 메모리에 불러오기, 인코딩 형식은 utf-8로 지정
train_text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

#텍스트가 총 몇 자인지를 확인
print('텍스트의 길이 : {} charaters'.format(len(train_text)))
print()

#처음에는 300자를 임시로 확인
print(train_text[:300])

#훈련 데이터입력 정제
# from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py

def clean_str(string):
  string = re.sub(r"[^가-힣A-Za-z0-9(),!?'\']"," ",string)
  string = re.sub(r"\'s", "\'s", string)
  string = re.sub(r"\'ve","\'ve",string)
  string = re.sub(r"n\'t", "n\'t",string)
  string = re.sub(r"\'re","\'re",string)
  string = re.sub(r"\'d","\'d",string)
  string = re.sub(r"\'ll","\'ll",string)
  string = re.sub(r",",",",string)
  string = re.sub(r"!"," ! ",string)
  string = re.sub(r"\(", " \( ",string)
  string = re.sub(r"\)", " \) ",string)
  string = re.sub(r"\?", " \? ",string)
  string = re.sub(r"\?", " \? ",string)
  string = re.sub(r"\s{2,}"," ",string)
  string = re.sub(r"\s{2,}","\'",string)
  string = re.sub(r"\'","",string)
  
  return string

train_text = train_text.split('\n')
train_text = [clean_str(sentence) for sentence in train_text]
train_text_X = []

for sentence in train_text:
  train_text_X.extend(sentence.split(' '))
  train_text_X.append('\n')

train_text_X = [word for word in train_text_X if word != '']

print(train_text_X[:20])

#단어를 토큰화 작업
#Tokenizer를 쓰면 계산시간이 늘기 때문에 직접 토큰화 진행

#단어 set생성
vocab = sorted(set(train_text_X))
vocab.append('UNK')
print('{} unique words '.format(len(vocab)))

#vocab list를 숫자로 매핑하고, 반대도 실행합니다
word2idx = {u:i for i, u in enumerate(vocab)}
idx2word = np.array(vocab)

text_as_int = np.array([word2idx[c] for c in train_text_X])

#word2idx의 일부를 알아보기 쉽게 출력
print('{')
for word, _ in zip(word2idx, range(10)):
  print('{:4s}: {:3d},'.format(repr(word), word2idx[word]))
print(' ...\n}')
#텍스트에 존재하지 않는 토큰을 나타내는 UNK 사용. 총 332640개의 단어중 UNK인덱스는 332639개
print('index of UNK: {}'.format(word2idx['UNK']))

print(train_text_X[:20])
print(text_as_int[:20])

# seq_length를 25로 설정해서 25개의 단어가 주어졌을때 다음 단어를 예측하도록 데이터를 만듬
seq_length = 25
examples_per_epoch = len(text_as_int)
#기존에 사용했던 train_X,train_Y를 넘파이 array로 만드는 방식 말고 tf.data.Dataset을 사용
#코드의 간편화, 데이터 섞기, 배치 수만큼 자르기, 매핑등의 장점
sentence_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
#drop_remainder=True을 사용해서 남는 부분은 삭제
#출력해서 의도한 대로 26개 단어 반환
sentence_dataset = sentence_dataset.batch(seq_length+1, drop_remainder=True)
for item in sentence_dataset.take(1):
  print(idx2word[item.numpy()])
  print(item.numpy())

def split_input_target(chunk):
  return [chunk[:-1], chunk[:-1]]

train_dataset = sentence_dataset.map(split_input_target)
for x,y in train_dataset.take(1):
  print(idx2word[x.numpy()])
  print(x.numpy())
  print(idx2word[y.numpy()])
  print(y.numpy())

#단어 단위 생성 모델정의
total_words = len(vocab)
model = tf.keras.Sequential([
                             tf.keras.layers.Embedding(total_words, 100, input_length=seq_length),
                             tf.keras.layers.LSTM(units=100, return_sequences=True),
                             tf.keras.layers.Dropout(0.2),
                             tf.keras.layers.LSTM(units=100),
                             tf.keras.layers.Dense(total_words, activation='softmax')
])

model.compile(optimizer='adam', loss= 'sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

#단어 단위 생성 모델 학습
from tensorflow.keras.preprocessing.sequence import pad_sequences

def testmodel(epoch, logs):
  if epoch % 5 !=0 and epoch !=49:
    return
  test_sentence = train_text[0]

  next_words=100
  for _ in range(next_words):
    test_text_X = text_sentence.split(' ')[-seq_length:]
    test_text_X = np.array([word2idx[c] if c in word2idx else word2idx['UNK'] for c in test_text_X])
    test_text_X = pad_sequences([test_text_X], maxlen = seq_length, padding = 'pre', value=word2idx['UNK'])
    output_idx = model.predict_classes(test_text_X)
    test_sentence += ' ' + idx2word[output_idx[0]]

  print()
  print(test_sentence)
  print()

testmodelcb = tf.keras.callbacks.LambdaCallback(on_epoch_end=testmodel)

history = model.fit(train_dataset.repeat(), epochs=50, steps_per_epoch = steps_per_epoch, callbacks=[testmodelcb], verbose=2)

from tensorflow.keras.preprocessing.sequence import pad_sequences
test_sentence = '동대문에 나가 공무 후 활을 쐈다'

next_words =100
for _ in range(next_words):
  test_text_X = test_sentence.split(' ')[-seq_length:]
  test_text_X = np.array([word2idx[c] if c in word2idx else total_words+1 for c in test_text_X])
  test_text_X = pad_sequences([test_text_X], maxlen=seq_length, padding = 'pre', value=word2idx['UNK'])

  output_idx = model.predict_classes(test_text_X)
  test_sentence += ' ' + idx2word[output_idx[0]]

print(test_sentece)

# 한글을 자소단위로 분리하고 다시 합칠 수 있는 라이브러리로 jamotools를 사용
# !pip install jamotools

import jamotools

train_text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
s = train_text[:100]
print(s)

#한글 텍스트를 자모 단위로 분리합니다. 한자 등에는 영향이 없습니다
s_split = jamotools.split_syllabels(s)
print(s_split)

# 자모를 분리했다가 다시 합친 s2는 원래 텍스트인 s와 같다는 것을 두번째 출력인 True인 것에서 확인할 수 있다
s2 = jamotools.join_jamos(s_split)
print(s2)
print(s == s2)

#자모 토큰화
#텍스트를 자모 단위로 나눕니다. 데이터가 크기 때문에 약간 시간이 걸립니다.
train_text_X = jamotools.split_syllabels(train_text)
vocab = sorted(set(train_text_X))
vocab.append('UNK')
print('{} unique charatoers'.format(len(vocab)))

#vocab list를 숫자로 매핑 그리고 역으로도 실행
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

#word2idx의 일부를 알아보기 쉽게 출력해봅니다.
print('{')
for char, _ in zip(char2idx, range(10)):
  print(' {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('.....\n}')

print('index of UNK: {}'.format(char2idx['UNK']))

#토큰 데이터 확인
print(train_text_X[:20])
print(text_as_int[:20])

#학습 데이터세트 생성
seq_length = 80
examples_per_epoch = len(text_as_int) //seq_length
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

char_dataset = char_dataset.batch(seq_length+1, drop_remainder=True)
for item in char_dataset.take(1):
  print(idx2char[item.numpy()])
  print(item.numpy())

def split_input_target(chunk):
  return [chunk[:-1], chunk[-1]]

train_dataset = char_dataset.map(split_input_target)
for x,y in train_dataset.take(1):
  print(idx2char[x.numpy()])
  print(x.numpy())
  print(idx2char[y.numpy()])
  print(y.numpy())

BATCH_SIZE = 256
steps_per_epoch = examples_per_epoch //BATCH_SIZE
BUFFER_SIZE = 10000

train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

#자소 단위 생성 모델 정의
total_chars = len(vocab)
model = tf.keras.Sequential([
                             tf.keras.layers.Embedding(total_chars, 100, input_length=seq_length),
                             tf.keras.layers.LSTM(units=400),
                             tf.keras.layers.Dense(total_chars, activation='softmax')
])

model.compile(optimizer='adma', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

from tensorflw.keras.preprocessing.sequence import pad_sequences

def testmodel(epoch, logs):
  if epoch % 5 != 0 and epoch != 99:
    return

  test_sentence = train_text[:48]
  test_sentence = jamotools.split_syllabels(test_sentence)

  next_chars = 300
  for _ in range(next_chars):
    test_text_X = test_sentence[-seq_length:]
    test_text_X = np.array([char2idx[c] if c in char2idx else char2idx['UNK'] for c in test_text_X])
    test_text_X = pad_sequences([test_text_X], maxlen=seq_length, padding ='pre', value=char2idx['UNK'])

    output_idx = model.predict_classes(test_text_X)
    test_sentence += idx2char[output_idx[0]]

  print()
  print(jamotools.join_jamos(test_sentence))
  print()

testmodelcb = tf.keras.callbacks.LambdaCallback(on_epoch_end=testmodel)

history = model.fit(train_dataset.repeat(), epochs=100, steps_per_epoch=steps_per_epoch, callbacks=[testmodelcb], verbose=2)

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.

텍스트의 길이 : 26265493 charaters

﻿태조 이성계 선대의 가계. 목조 이안사가 전주에서 삼척·의주를 거쳐 알동에 정착하다 
태조 강헌 지인 계운 성문 신무 대왕(太祖康獻至仁啓運聖文神武大王)의 성은 이씨(李氏)요, 휘(諱)는 단(旦)이요, 자(字)는 군진(君晉)이다. 그전의 휘(諱)는 이성계(李成桂)요, 호(號)는 송헌(松軒)이다. 전주(全州)의 대성(大姓)이다. 사공(司空) 휘(諱) 이한(李翰)이 신라(新羅)에 벼슬하여 태종왕(太宗王)001) 의 10대(代) 손자인 군윤(軍尹) 김은의(金殷義)의 딸에게 장가들어 시중(侍中) 휘(諱) 이자연(李自延)을 낳았다. 시중이 
['태조', '이성계', '선대의', '가계', '목조', '이안사가', '전주에서', '삼척', '의주를', '거쳐', '알동에', '정착하다', '\n', '태조', '강헌', '지인', '계운', '성문', '신무', '대왕']
352275 unique words 
{
'\n':   0,
'!' :   1,
',' :   2,
',광연전':   3,
',동해':   4,
',박강':   5,
',사배':   6,
',연':   7,
',흥':   8,
'001':   9,
 ...
}
index of UNK: 352274
['태조', '이성계', '선대의', '가계', '목조', '이안사가', '전주에서', '삼척', '의주를', '거쳐', '알동에', '정착하다', '\n', '태조', '강헌', '지인', '계운', '성문', '신무', '대왕']
[317209 243792 172112  19043 118674 244465 266449 165427 239355  31342
 202515 271742      0 31

NameError: ignored